# advent of code 2024 - [day 18](https://adventofcode.com/2024/day/18)

### Parsing

In [1]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

In [3]:
def parse(file='input.txt'):
    file = open(file, 'r')
    for _, line in enumerate(file):
        yield tuple([int(x) for x in line.strip().split(',')])

In [4]:
filename, size, kilobyte = 'input.txt', 70, 1024
#filename, size, kilobyte = 'test.txt', 6, 12

In [ ]:
from itertools import islice

def parse_n_first(filename, n_first):
    for x,y in islice(parse(filename), 0, n_first):
        yield (x,y)

In [6]:
clean_queries = [
"CALL apoc.schema.assert({},{});",
"""MATCH (n)
CALL {WITH n DETACH DELETE n}
IN TRANSACTIONS OF 1000 ROWS;""",
"""CALL gds.graph.list()
YIELD graphName
WITH graphName AS g
CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
WITH graphName RETURN graphName;"""]

for q in clean_queries:
    gds.run_cypher(q, {})

In [ ]:
gds.run_cypher('CREATE INDEX tile_x IF NOT EXISTS FOR (r:Tile) ON (r.x)')
gds.run_cypher('CREATE INDEX tile_y IF NOT EXISTS FOR (r:Tile) ON (r.y)')
gds.run_cypher('CREATE INDEX tile_valid_until IF NOT EXISTS FOR (r:Tile) ON (r.valid_until)')

In [8]:

build_q= ["""
UNWIND range(0,$size) AS x
UNWIND range(0,$size) AS y
CREATE (:Tile {x:x, y:y, X:x, Y:-y})
""",
"""
UNWIND range(0,$size) AS x
MATCH (t:Tile {x:x})
WITH x, t ORDER BY t.y
WITH x, collect(t) AS col
CALL apoc.nodes.link(col,'S')
""",
"""
UNWIND range(0,$size) AS y
MATCH (t:Tile {y:y})
WITH y, t ORDER BY t.x
WITH y, collect(t) AS col
CALL apoc.nodes.link(col,'E')
""",
"""
MATCH (a)-[:E]->(b)
MERGE (b)-[:W]->(a)
""","""
MATCH (a)-[:S]->(b)
MERGE (b)-[:N]->(a)
"""

]
for q in build_q:
    gds.run_cypher(q, {'size':size})

In [ ]:
corrupted_q = """
UNWIND $kb AS byte
MATCH (t:Tile {x:byte.x, y:byte.y})
SET t:Corrupted
"""

gds.run_cypher(corrupted_q, {'kb':[{"t":t, "x":x, "y":y} for t, (x, y) in enumerate(parse_n_first(filename, kilobyte))]})


In [10]:
st_query = ["""
MATCH (t:Tile {x:0, y:0})
SET t:Source
""",
"""
MATCH (t:Tile {x:$size, y:$size})
SET t:Target
"""]
for q in st_query:
    gds.run_cypher(q, {'size':size})

In [ ]:
gds.run_cypher("""MATCH (source:Tile&!Corrupted)-[r:N|S|E|W]->(target:Tile&!Corrupted)
RETURN gds.graph.project(
  'myGraph',
  source,
  target,
  {}
)""")

In [ ]:
gds.run_cypher("""
MATCH (source:Source), (target:Target)
CALL gds.shortestPath.dijkstra.stream('myGraph', {
    sourceNode: source,
    targetNodes: target
})
YIELD totalCost
RETURN toInteger(totalCost) AS part1
""")

In [13]:
part2_prep_queries = ["""
MATCH (t:Corrupted) REMOVE t:Corrupted
""",
"""
UNWIND $kb AS byte
MATCH (t:Tile {x:byte.x, y:byte.y})
SET t.valid_until = byte.t
"""]
for q in part2_prep_queries:
    gds.run_cypher(q, {'kb':[{"t":t, "x":x, "y":y} for t, (x, y) in enumerate(parse(filename))]})

In [ ]:
final_part2_q = """
UNWIND range(0,size($kb)-1) AS t
MATCH path = SHORTEST 1 (:Source)(()-->(jx:Tile WHERE jx.valid_until IS null OR jx.valid_until > t))*(:Target)
WITH t, path ORDER BY t DESC LIMIT 1
MATCH (x:Tile {valid_until:t+1})
RETURN t, toString(x.x)+','+toString(x.y) AS part2
"""


gds.run_cypher(final_part2_q, {'kb':[{"t":t, "x":x, "y":y} for t, (x, y) in enumerate(parse(filename))]})